In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
width 
length 
height


x1 = np.random.rand(-width/2 , width/2)
y1 = np.random.rand(0 , length)


x2 = np.random.rand(-width/2 , width/2)
y2 = np.random.rand(0 , length)

L = np.sqrt((x1-y1)**2 + (x2-y2)**2 + height**2)


cos = h/L

v_signal \approx 2/3 c 

t_signal = (np.sqrt(x1**2 + y1**2) - np.sqrt(x2**2 + y2**2))/v_signal

t = L/v_mu  + t_signal + t_0

prob of event = cos**2